In [77]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import random
from bs4 import BeautifulSoup              # Library to parse data from webpages
driver = webdriver.Chrome('c:/Users/TV/Downloads/chromedriver_win32/chromedriver.exe')

In [10]:
# unsure if needed
#windows defender firewall --> allow app or feature --> change settings --> python.exe (public)
#when operating on USF network

In [78]:
#remove the location filter by default
#click on the 'x' for the filter button to remove it
def removeFilter():
    try:
        driver.find_element_by_class_name("filter-drp-close").click()
    except NoSuchElementException:
        pass

In [74]:
#collect desired info from bank listing
#set as function to pull from each page
def getData():
    for bank in soup.find_all(class_="inst-data"):
        name = bank.find(attrs={'itemprop':'name'})['content']
        reviews = bank.find(class_="rev-count").get_text()
        reviews = int(reviews.replace(',',''))
        link = bank.find(attrs={'itemprop':'url'})['content']
        dict['Name'].append(name)
        dict['Reviews'].append(reviews)
        dict['Profile Link'].append(link)

In [116]:
#declare dictionary
dict = {'Name':[],
        'Reviews':[],
        'Profile Link':[]}

In [117]:
#from 1 to 10 (# of webpages)
for i in range(1,11):
    url = 'https://wallethub.com/banks#pg={}'.format(i)
    time.sleep(random.randint(1,5)) #wait random amount of time in between pages
    driver.get(url)
    
    if (i==1):
        removeFilter()
        
    time.sleep(1)                              # Give Javascript time to render
    soup = BeautifulSoup(driver.page_source)
    getData()

In [126]:
#take collected info from dict
# and convert to dataframe
import pandas as pd
df = pd.DataFrame(dict)

In [138]:
#add column showing whether big bank or not

# big banks based on
# https://en.wikipedia.org/wiki/List_of_largest_banks_in_the_United_States and
# https://www.federalreserve.gov/newsevents/pressreleases/bcreg20201217a.htm
isBig = []
bigBank = ['Chase','Bank of America','Citibank','Wells Fargo']
for bank in df['Name']:
    if bank in bigBank:
        val = 'yes'
    else:
        val = 'no'
    isBig.append(val)
df['Big Bank?'] = isBig

In [150]:
#show big banks first, then those with most reviews
df.sort_values(by=['Big Bank?','Reviews'],inplace=True, ascending=False)
df.reset_index(drop=True,inplace=True)

In [162]:
#reorder columns to set profile link as last
#view df
n = len(df)
df = df.iloc[0:n,[0, 1, 3, 2]]
df.style.format({
    "Reviews": "{:,}",
}).hide_index()

Name,Reviews,Big Bank?,Profile Link
Bank of America,"39,327",yes,https://wallethub.com/profile/bank-of-america-13000450i
Citibank,"30,738",yes,https://wallethub.com/profile/citibank-13001291i
Chase,"25,726",yes,https://wallethub.com/profile/chase-13001251i
Wells Fargo,"11,170",yes,https://wallethub.com/profile/wells-fargo-13007950i
Capital One,"72,492",no,https://wallethub.com/profile/capital-one-13001087i
U.S. Bank,"21,732",no,https://wallethub.com/profile/us-bank-13007637i
PNC,"6,807",no,https://wallethub.com/profile/pnc-13005045i
First National Bank of Omaha,"6,023",no,https://wallethub.com/profile/first-national-bank-of-omaha-13003011i
Commerce Bank,"4,089",no,https://wallethub.com/profile/commerce-bank-13001576i
TD Bank,"3,806",no,https://wallethub.com/profile/td-bank-13006307i


In [179]:
#take a sample of the top 12 rows
df2 = df.copy()
df2 = df.loc[0:11]
df2

,Name,Reviews,Big Bank?,Profile Link
0,Bank of America,39327,yes,https://wallethub.com/profile/bank-of-america-...
1,Citibank,30738,yes,https://wallethub.com/profile/citibank-13001291i
2,Chase,25726,yes,https://wallethub.com/profile/chase-13001251i
3,Wells Fargo,11170,yes,https://wallethub.com/profile/wells-fargo-1300...
4,Capital One,72492,no,https://wallethub.com/profile/capital-one-1300...
5,U.S. Bank,21732,no,https://wallethub.com/profile/us-bank-13007637i
6,PNC,6807,no,https://wallethub.com/profile/pnc-13005045i
7,First National Bank of Omaha,6023,no,https://wallethub.com/profile/first-national-b...
8,Commerce Bank,4089,no,https://wallethub.com/profile/commerce-bank-13...
9,TD Bank,3806,no,https://wallethub.com/profile/td-bank-13006307i


In [180]:
#of these rows, find (manually, for now) the number of written reviews
#based on pages available on profile page (as each page has 10 reviews, ignoring replies)
#note: rough estimate as last page may have less than 10 reviews
link_vals=[345,405,187,862,110,102,25,9,91,21,56,9]
link_vals = [x*10 for x in link_vals]
df2['Written Reviews'] = link_vals

C:\Users\TV\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [181]:
#show big banks first, then order by written reviews
df2 = df2.iloc[0:n,[0, 1, 4, 2, 3]]
df2.sort_values(by=['Big Bank?','Written Reviews'],inplace=True, ascending=False)
df2.style.format({
    "Reviews": "{:,}",
    "Written Reviews":"{:,}",
}).hide_index()

Name,Reviews,Written Reviews,Big Bank?,Profile Link
Wells Fargo,"11,170","8,620",yes,https://wallethub.com/profile/wells-fargo-13007950i
Citibank,"30,738","4,050",yes,https://wallethub.com/profile/citibank-13001291i
Bank of America,"39,327","3,450",yes,https://wallethub.com/profile/bank-of-america-13000450i
Chase,"25,726","1,870",yes,https://wallethub.com/profile/chase-13001251i
Capital One,"72,492","1,100",no,https://wallethub.com/profile/capital-one-13001087i
U.S. Bank,"21,732","1,020",no,https://wallethub.com/profile/us-bank-13007637i
Commerce Bank,"4,089",910,no,https://wallethub.com/profile/commerce-bank-13001576i
Citizens Bank,"3,262",560,no,https://wallethub.com/profile/citizens-bank-13005518i
PNC,"6,807",250,no,https://wallethub.com/profile/pnc-13005045i
TD Bank,"3,806",210,no,https://wallethub.com/profile/td-bank-13006307i
